In [82]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [83]:
df = pd.read_csv('Data/Listening_Data/songs_listened_to_with_all_info_unique.csv')
df = df.drop('Unnamed: 0.1', axis=1)
df = df.drop('Unnamed: 0', axis = 1)
df.head(2)

,endTime,artistName,trackName,albumName,msPlayed,month,year,Season,trackUri,danceability,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,1/20/2022 4:50,Lonr.,Make the Most (feat. H.E.R.),Land Of Nothing Real,60,1,2022,1,spotify:track:6UF1vUIoLleFrhcxg2L26V,0.566,...,0.0806,0.838,78.743,audio_features,6UF1vUIoLleFrhcxg2L26V,spotify:track:6UF1vUIoLleFrhcxg2L26V,https://api.spotify.com/v1/tracks/6UF1vUIoLleF...,https://api.spotify.com/v1/audio-analysis/6UF1...,213851,4
1,1/20/2022 4:53,Lonr.,Make the Most (feat. H.E.R.),Land Of Nothing Real,213851,1,2022,1,spotify:track:6UF1vUIoLleFrhcxg2L26V,0.566,...,0.0806,0.838,78.743,audio_features,6UF1vUIoLleFrhcxg2L26V,spotify:track:6UF1vUIoLleFrhcxg2L26V,https://api.spotify.com/v1/tracks/6UF1vUIoLleF...,https://api.spotify.com/v1/audio-analysis/6UF1...,213851,4


In [84]:
df['Percentage_Listened'] = (df['msPlayed'] / df['duration_ms']) * 100

In [85]:
df = df[['month', 'danceability', 'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'Percentage_Listened']]
df.head(2)

,month,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,Percentage_Listened
0,1,0.566,0.693,10,-6.917,0.368,0.361,0.0,0.0806,0.838,78.743,0.028057
1,1,0.566,0.693,10,-6.917,0.368,0.361,0.0,0.0806,0.838,78.743,100.000000


In [86]:
# df = df[['tempo', 'Percentage_Listened']]

In [87]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

In [88]:
scaler = MinMaxScaler()
scaler.fit(df)
X=scaler.transform(df)


In [89]:
inertia = []
for i in range(1,11):
    kmeans = KMeans(
        n_clusters=i, init="k-means++",
        n_init=10,
        tol=1e-04, random_state=42
    )
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)
fig = go.Figure(data=go.Scatter(x=np.arange(1,11),y=inertia))
fig.update_layout(title="Inertia vs Cluster Number",xaxis=dict(range=[0,11],title="Cluster Number"),
                  yaxis={'title':'Inertia'},
                 annotations=[
        dict(
            x=4,
            y=inertia[3],
            xref="x",
            yref="y",
            text="Elbow!",
            showarrow=True,
            arrowhead=7,
            ax=20,
            ay=-40
        )
    ])

Looks like the elbow is created around K = 3-4. So lets use 4!

Note: Inertia can be recognized as a measure of how internally coherent clusters are.

Inertia measures how well a dataset was clustered by K-Means. It is calculated by measuring the distance between each data point and its centroid, squaring this distance, and summing these squares across one cluster. A good model is one with low inertia AND a low number of clusters ( K )

In [90]:
kmeans = KMeans(
        n_clusters=4, init="k-means++",
        n_init=10,
        tol=1e-04, random_state=42
    )
kmeans.fit(X)

KMeans(n_clusters=4, random_state=42)

In [91]:

clusters=pd.DataFrame(X,columns=df.columns)
clusters['label']=kmeans.labels_
polar=clusters.groupby("label").mean().reset_index()
polar=pd.melt(polar,id_vars=["label"])
fig = px.line_polar(polar, r="value", theta="variable", color="label", line_close=True,height=800,width=1400)
fig.show()

In [92]:
pie=clusters.groupby('label').size().reset_index()
pie.columns=['label','value']
px.pie(pie,values='value',names='label',color=['blue','red','green', 'purple'])